# Revenue-Based Loan in Marlowe

*Executive Summary*

This revenue-based loan involves variable payments that are computed as a fixed percentage of the borrower's revenue during each period of the loan. The payments continue until principal plus interest is paid off. There is no penalty for missing a payment. An oracle reports the borrower's revenue each period, prior to the borrower depositing that amount. The contract was both simulated and run on the Cardano blockchain: that involved 8 transactions.

[This video](https://vimeo.com/726500312/f90fd85ed7) shows [the contract](revenue-based-loan.hs) being run in Marlowe Playground.

*Sequences of transactions in this example*

1.  The oracle creates the contract.
2.  The lender deposits the principal.
3.  The oracle reports the borrower's revenue in the first period.
4.  The borrower deposits their first payment.
5.  The oracle reports the borrower's revenue in the second period.
6.  The borrower deposits their second payment.
7.  The oracle reports the borrower's revenue in the third period.
8.  The borrower deposits their third payment, completing the repayment of principal plus interest.

## Preliminaries

Make a note of the versions of the Marlowe repository and software.

In [ ]:
marlowe-cli --version

marlowe-cli 0.0.5.0


In [ ]:
cardano-cli --version

cardano-cli 1.34.1 - linux-x86_64 - ghc-8.10
git rev 0000000000000000000000000000000000000000


In [ ]:
git rev-parse HEAD

de3ca53f994f989ba7f5693e5e03bdfbcd52beee


### Select the Cardano network

In [1]:
export CARDANO_TESTNET_MAGIC=1567
export CARDANO_NODE_SOCKET_PATH=node.socket

### Set up keys, addresses, and tokens

This section can be omitted if only off-chain simulation is performed.

We use public key hashes instead of role tokens to identify the participants in the contract.

### Djed

The contract will use Djed for payments.

In [2]:
DJED_POLICY=1f4cc97451944ce039366dee5d9acd36a2784dd22e7c3431e50264e8
DJED_NAME=DjedUSD
DJED=$DJED_POLICY.$DJED_NAME
echo $DJED

1f4cc97451944ce039366dee5d9acd36a2784dd22e7c3431e50264e8.DjedUSD


In `cardano-cli` the token name is printed in hexadecimal, so we have . . .

In [3]:
echo $DJED_POLICY.$(echo -n $DJED_NAME | basenc --base16 | tr '[:upper:]' '[:lower:]')

1f4cc97451944ce039366dee5d9acd36a2784dd22e7c3431e50264e8.446a6564555344


### Parties to the Contract

#### The Oracle: Christopher Marlowe

In [4]:
MARLOWE_SKEY=christopher-marlowe.skey
MARLOWE_VKEY=christopher-marlowe.vkey
MARLOWE_PKH=$(cardano-cli address key-hash --payment-verification-key-file $MARLOWE_VKEY)
MARLOWE_ADDR=$(cat christopher-marlowe.testnet.address)
echo $MARLOWE_ADDR
echo $MARLOWE_PKH

addr_test1vrssw4edcts00kk6lp7p5n64666m23tpprqaarmdwkaq69gfvqnpz
e107572dc2e0f7dadaf87c1a4f55d6b5b5456108c1de8f6d75ba0d15


Christopher Marlowe has these UTxOs at his address:

In [5]:
cardano-cli query utxo --testnet-magic $CARDANO_TESTNET_MAGIC --address $MARLOWE_ADDR

                           TxHash                                 TxIx        Amount
--------------------------------------------------------------------------------------
f872d293f9097a5e9b28e4b5817ce377544a179a6813282b6d2604fc4aed0fb6     0        994743733 lovelace + TxOutDatumNone


Make a note of this UTxO for later use.

In [6]:
TX_0_MARLOWE="f872d293f9097a5e9b28e4b5817ce377544a179a6813282b6d2604fc4aed0fb6#0"

#### The Lender: Francis Beaumont

In [7]:
BEAUMONT_SKEY=francis-beaumont.skey
BEAUMONT_VKEY=francis-beaumont.vkey
BEAUMONT_PKH=$(cardano-cli address key-hash --payment-verification-key-file $BEAUMONT_VKEY)
BEAUMONT_ADDR=$(cat francis-beaumont.testnet.address)
echo $BEAUMONT_ADDR
echo $BEAUMONT_PKH

addr_test1vzzpzll6gsl9npf8wfhk2zg8sy2we50jcqc7w8w46gua2pqq7cw2q
84117ffa443e598527726f6509078114ecd1f2c031e71dd5d239d504


In [8]:
cardano-cli query utxo --testnet-magic $CARDANO_TESTNET_MAGIC --address $BEAUMONT_ADDR

                           TxHash                                 TxIx        Amount
--------------------------------------------------------------------------------------
923e24f6980b547ec3fe76febd97aa050d8d4e52111ed69e9de0ef80a7cd2f93     0        970562214 lovelace + TxOutDatumNone
923e24f6980b547ec3fe76febd97aa050d8d4e52111ed69e9de0ef80a7cd2f93     1        10000000 lovelace + 1000 1f4cc97451944ce039366dee5d9acd36a2784dd22e7c3431e50264e8.446a6564555344 + TxOutDatumNone


Make a note of these UTxOs for later use.

In [9]:
TX_0_BEAUMONT_ADA="923e24f6980b547ec3fe76febd97aa050d8d4e52111ed69e9de0ef80a7cd2f93#0"
TX_0_BEAUMONT_DJED="923e24f6980b547ec3fe76febd97aa050d8d4e52111ed69e9de0ef80a7cd2f93#1"

#### The Borrower: Thomas Kyd

In [10]:
KYD_SKEY=thomas-kyd.skey
KYD_VKEY=thomas-kyd.vkey
KYD_PKH=$(cardano-cli address key-hash --payment-verification-key-file $KYD_VKEY)
KYD_ADDR=$(cat thomas-kyd.testnet.address)
echo $KYD_ADDR
echo $KYD_PKH

addr_test1vr7n0zzth5zycuh972w7rdmh48qur4f3wu6ntn2m2h30dlcvltuy5
fd37884bbd044c72e5f29de1b777a9c1c1d531773535cd5b55e2f6ff


Thomas Kyd has these UTxOs at his address:

In [11]:
cardano-cli query utxo --testnet-magic $CARDANO_TESTNET_MAGIC --address $KYD_ADDR

                           TxHash                                 TxIx        Amount
--------------------------------------------------------------------------------------
625b9f53ee6f5f05ca54834b99ca3dceb097be8e934828d353acb2a7e6c74cb7     0        1002574041 lovelace + TxOutDatumNone
923e24f6980b547ec3fe76febd97aa050d8d4e52111ed69e9de0ef80a7cd2f93     2        10000000 lovelace + 100 1f4cc97451944ce039366dee5d9acd36a2784dd22e7c3431e50264e8.446a6564555344 + TxOutDatumNone


Make a note of these UTxOs for later use.

In [12]:
TX_0_KYD_ADA="625b9f53ee6f5f05ca54834b99ca3dceb097be8e934828d353acb2a7e6c74cb7#0"
TX_0_KYD_DJED="923e24f6980b547ec3fe76febd97aa050d8d4e52111ed69e9de0ef80a7cd2f93#2"

### Set up the contract

Start the contract at the current time.

In [13]:
NOW=$(($(date -u +%s) * 1000))
echo $NOW

1656878706000


Use a four hour time window for submitting transactions. This is just for convenience when testing this contract.

In [14]:
HOUR=$((60 * 60 * 1000))
DAY=$((24 * HOUR))
LATER=$((NOW + 4 * HOUR))
echo $LATER

1656893106000


There is a minimum Ada requirement associated with a native token.

In [15]:
ADA=1000000
MIN_ADA=$((2 * ADA))
echo $MIN_ADA

2000000


Create a Haskell program that will generate a contract with six rounds of bidding for three bidders. We could use `runhaskell` instead of `ghc`, but `runhaskell` is slow because it doesn't perform optimizations.

In [16]:
PRINCIPAL=1000
INTEREST=100
RATE=3

In [17]:
runhaskell << EOI

{-# LANGUAGE OverloadedStrings #-}
module RevenueBasedLoan where

import Data.Aeson (encodeFile)
import Language.Marlowe.Extended.V1
import qualified Language.Marlowe.Core.V1.Semantics.Types as S (Contract)

main :: IO ()
main =
  encodeFile "revenue-based-loan-01.contract"
    . (toCore :: Contract -> Maybe S.Contract)
    $ contract

djed :: Token
djed = Token "$DJED_POLICY" "DjedUSD"

lender :: Party
lender = PK "$BEAUMONT_PKH"

borrower :: Party
borrower = PK "$KYD_PKH"

oracle :: Party
oracle = PK "$MARLOWE_PKH"

revenueChoice :: ChoiceId
revenueChoice = ChoiceId "Revenue" oracle

revenueValue :: Value
revenueValue = ChoiceValue revenueChoice

revenueBounds :: Bound
revenueBounds = Bound 1 1000000

principal :: Value
principal = Constant $PRINCIPAL

interest :: Value
interest = Constant $INTEREST

paymentPercent :: Value
paymentPercent = Constant $RATE

remainingDue :: Value
remainingDue = UseValue "Remaining Due"

nextPayment :: Value
nextPayment = UseValue "Next Payment"

makePayment :: Integer -> Contract -> Contract
makePayment period continuation =
  When
    [
      Case (Choice revenueChoice [revenueBounds])
        $ Let "Next Payment" (DivValue (MulValue revenueValue paymentPercent) (Constant 100))
        $ If (ValueGT nextPayment remainingDue)
            (
              When
                [
                  Case (Deposit lender borrower djed remainingDue)
                    $ Pay lender (Party lender) djed remainingDue
                      Close  
                ]
                (POSIXTime $ $NOW + (period + 1) * $DAY)
                continuation
            )
            (
              When
                [
                  Case (Deposit lender borrower djed nextPayment) 
                    $ Pay lender (Party lender) djed nextPayment
                    $ Let "Remaining Due" (SubValue remainingDue nextPayment)
                      continuation  
                ]
                (POSIXTime $ $NOW + (period + 1) * $DAY)
                continuation
            )
    ]
    (POSIXTime $ $NOW + (period + 1) * $DAY - 1 * $HOUR)
    continuation

collectRemainder :: Contract
collectRemainder =
  When
    [
      Case (Deposit lender borrower djed remainingDue)
        Close
    ]
    (POSIXTime $((NOW + 1000 * DAY)))
    Close

contract :: Contract
contract =
  When
    [
      Case (Deposit lender lender djed principal)
        $ Pay lender (Party borrower) djed principal
        $ Let "Remaining Due" (AddValue principal interest)
        $ foldr makePayment collectRemainder [1..5]
    ]
    (POSIXTime $((NOW + 1 * DAY)))
    Close
    
EOI

How large is the contract?

In [18]:
ls -lh revenue-based-loan-01.contract

-rw-rw-r-- 1 bbush bbush-upg 1.1M Jul  3 14:05 revenue-based-loan-01.contract


How many `Case` statements does it contain?

In [19]:
sed -e 's/case/&\n/g' revenue-based-loan-01.contract | grep case | wc -l

2048


### Set the initial state

The Seller will deposit 2 Ada when the contract is created.

In [20]:
yaml2json << EOI > revenue-based-loan-01.state
accounts:
- - - pk_hash: $MARLOWE_PKH
    - currency_symbol: ''
      token_name: ''
  - $MIN_ADA
boundValues: []
choices: []
minTime: $NOW
EOI

In [21]:
cat revenue-based-loan-01.state

{"accounts":[[[{"pk_hash":"e107572dc2e0f7dadaf87c1a4f55d6b5b5456108c1de8f6d75ba0d15"},{"currency_symbol":"","token_name":""}],2000000]],"boundValues":[],"choices":[],"minTime":1656878706000}


## Simulate the Contract

We start by simulating the contract before running on the blockchain. This lets us test our inputs without committing any Ada. It also separates the logic of the contract's progression from the details of eUTxO management.

### 1. Initialize the contract information

Create a `.marlowe` file that contains all of the information needed to start the contract. Note that we merkleization the contract because of its large size.

In [22]:
marlowe-cli run initialize --contract-file revenue-based-loan-01.contract \
                           --state-file    revenue-based-loan-01.state    \
                           --out-file      revenue-based-loan-01.marlowe  \
                           --merkleize                                    \
                           --print-stats


Validator size: 12582
Base-validator cost: ExBudget {exBudgetCPU = ExCPU 24562825, exBudgetMemory = ExMemory 82600}


How large is the merkleized contract?

In [23]:
ls -lh revenue-based-loan-01.marlowe

-rw-rw-r-- 1 bbush bbush-upg 123K Jul  3 14:06 revenue-based-loan-01.marlowe


How many merkle hashes does it contain?

In [24]:
jq '.continuations | length' revenue-based-loan-01.marlowe

12


The minimum Ada deposit has been recorded.

In [25]:
jq .state revenue-based-loan-01.marlowe | json2yaml

accounts:
- - - pk_hash: e107572dc2e0f7dadaf87c1a4f55d6b5b5456108c1de8f6d75ba0d15
    - currency_symbol: ''
      token_name: ''
  - 2000000
boundValues: []
choices: []
minTime: 1656878706000


### 2. The lender deposits the principal

In [26]:
marlowe-cli run prepare --deposit-account PK=$BEAUMONT_PKH           \
                        --deposit-party   PK=$BEAUMONT_PKH           \
                        --deposit-token "$DJED"                      \
                        --deposit-amount $PRINCIPAL                  \
                        --invalid-before $NOW                        \
                        --invalid-hereafter $LATER                   \
                        --marlowe-file revenue-based-loan-01.marlowe \
                        --out-file     revenue-based-loan-02.marlowe \
                        --print-stats


Datum size: 827
Payment 1
  Acccount: PK "84117ffa443e598527726f6509078114ecd1f2c031e71dd5d239d504"
  Payee: Party (PK "fd37884bbd044c72e5f29de1b777a9c1c1d531773535cd5b55e2f6ff")
  Ada: 0.000000
  1f4cc97451944ce039366dee5d9acd36a2784dd22e7c3431e50264e8."DjedUSD": 1000


Now the contract still contains the initial Ada, but the principal has been paid to the borrower. The remaining payments due has been calculated.

In [27]:
jq .contract revenue-based-loan-01.marlowe | json2yaml

timeout: 1656965106000
timeout_continuation: close
when:
- case:
    deposits: 1000
    into_account:
      pk_hash: 84117ffa443e598527726f6509078114ecd1f2c031e71dd5d239d504
    of_token:
      currency_symbol: 1f4cc97451944ce039366dee5d9acd36a2784dd22e7c3431e50264e8
      token_name: DjedUSD
    party:
      pk_hash: 84117ffa443e598527726f6509078114ecd1f2c031e71dd5d239d504
  merkleized_then: a80c752321932efccbca109a5d1a394dfff6b9c7456ddff8efcafd8933dc7203


In [28]:
jq .state revenue-based-loan-02.marlowe | json2yaml

accounts:
- - - pk_hash: e107572dc2e0f7dadaf87c1a4f55d6b5b5456108c1de8f6d75ba0d15
    - currency_symbol: ''
      token_name: ''
  - 2000000
boundValues:
- - Remaining Due
  - 1100
choices: []
minTime: 1656878706000


### 3. The oracle reports the borrower's revenue in the first period

In [29]:
REVENUE_1=15000

In [30]:
marlowe-cli run prepare --choice-party PK=$MARLOWE_PKH               \
                        --choice-name "Revenue"                      \
                        --choice-number $REVENUE_1                   \
                        --invalid-before $NOW                        \
                        --invalid-hereafter $LATER                   \
                        --marlowe-file revenue-based-loan-02.marlowe \
                        --out-file     revenue-based-loan-03.marlowe \
                        --print-stats


Datum size: 970


The revenue value has been recorded and the next payment has been calculated.

In [31]:
jq .state revenue-based-loan-03.marlowe | json2yaml

accounts:
- - - pk_hash: e107572dc2e0f7dadaf87c1a4f55d6b5b5456108c1de8f6d75ba0d15
    - currency_symbol: ''
      token_name: ''
  - 2000000
boundValues:
- - Remaining Due
  - 1100
- - Next Payment
  - 450
choices:
- - choice_name: Revenue
    choice_owner:
      pk_hash: e107572dc2e0f7dadaf87c1a4f55d6b5b5456108c1de8f6d75ba0d15
  - 15000
minTime: 1656878706000


### 4. The borrower deposits their first payment

The borrower must deposit the next payment of 450 `DjedUSD` into the lender's account.

In [32]:
DEPOSIT_1=$((REVENUE_1 * RATE / 100))
echo $DEPOSIT_1

450


In [33]:
marlowe-cli run prepare --deposit-account PK=$BEAUMONT_PKH           \
                        --deposit-party PK=$KYD_PKH                  \
                        --deposit-amount $DEPOSIT_1                  \
                        --deposit-token $DJED                        \
                        --invalid-before $NOW                        \
                        --invalid-hereafter $LATER                   \
                        --marlowe-file revenue-based-loan-03.marlowe \
                        --out-file     revenue-based-loan-04.marlowe \
                        --print-stats


Warnings:
  TransactionShadowing "Remaining Due" 1100 650
Datum size: 781
Payment 1
  Acccount: PK "84117ffa443e598527726f6509078114ecd1f2c031e71dd5d239d504"
  Payee: Party (PK "84117ffa443e598527726f6509078114ecd1f2c031e71dd5d239d504")
  Ada: 0.000000
  1f4cc97451944ce039366dee5d9acd36a2784dd22e7c3431e50264e8."DjedUSD": 450


The remaining payments due has been updated.

In [34]:
jq .state.boundValues revenue-based-loan-04.marlowe | json2yaml

- - Remaining Due
  - 650
- - Next Payment
  - 450


### 5. The oracle reports the borrower's revenue in the second period

In [35]:
REVENUE_2=12000

In [36]:
marlowe-cli run prepare --choice-party PK=$MARLOWE_PKH               \
                        --choice-name "Revenue"                      \
                        --choice-number $REVENUE_2                   \
                        --invalid-before $NOW                        \
                        --invalid-hereafter $LATER                   \
                        --marlowe-file revenue-based-loan-04.marlowe \
                        --out-file     revenue-based-loan-05.marlowe \
                        --print-stats


Warnings:
  TransactionShadowing "Next Payment" 450 360
Datum size: 855


The next payment has been calculated.

In [37]:
jq .state.boundValues revenue-based-loan-05.marlowe | json2yaml

- - Remaining Due
  - 650
- - Next Payment
  - 360


### 6. The borrower deposits the second payment

In [38]:
DEPOSIT_2=$((REVENUE_2 * RATE / 100))
echo $DEPOSIT_2

360


In [39]:
marlowe-cli run prepare --deposit-account PK=$BEAUMONT_PKH           \
                        --deposit-party PK=$KYD_PKH                  \
                        --deposit-amount $DEPOSIT_2                  \
                        --deposit-token $DJED                        \
                        --invalid-before $NOW                        \
                        --invalid-hereafter $LATER                   \
                        --marlowe-file revenue-based-loan-05.marlowe \
                        --out-file     revenue-based-loan-06.marlowe \
                        --print-stats


Warnings:
  TransactionShadowing "Remaining Due" 650 290
Datum size: 666
Payment 1
  Acccount: PK "84117ffa443e598527726f6509078114ecd1f2c031e71dd5d239d504"
  Payee: Party (PK "84117ffa443e598527726f6509078114ecd1f2c031e71dd5d239d504")
  Ada: 0.000000
  1f4cc97451944ce039366dee5d9acd36a2784dd22e7c3431e50264e8."DjedUSD": 360


The remaining payments due has been calculated.

In [40]:
jq .state.boundValues revenue-based-loan-06.marlowe | json2yaml

- - Remaining Due
  - 290
- - Next Payment
  - 360


### 7. The oracle reports the borrower's revenue in the third period

In [41]:
REVENUE_3=19000

In [42]:
marlowe-cli run prepare --choice-party PK=$MARLOWE_PKH               \
                        --choice-name "Revenue"                      \
                        --choice-number $REVENUE_3                   \
                        --invalid-before $NOW                        \
                        --invalid-hereafter $LATER                   \
                        --marlowe-file revenue-based-loan-06.marlowe \
                        --out-file     revenue-based-loan-07.marlowe \
                        --print-stats


Warnings:
  TransactionShadowing "Next Payment" 360 570
Datum size: 741


The remaining due is now smaller than the payment based on the third period's revenue.

In [43]:
jq .state.boundValues revenue-based-loan-07.marlowe | json2yaml

- - Remaining Due
  - 290
- - Next Payment
  - 570


### 8. The borrower deposits the third payment

By paying the remaining due, the borrower completes the loan.

In [44]:
DEPOSIT_3=$((PRINCIPAL + INTEREST - DEPOSIT_1 - DEPOSIT_2))
echo $DEPOSIT_3

290


In [45]:
marlowe-cli run prepare --deposit-account PK=$BEAUMONT_PKH           \
                        --deposit-party PK=$KYD_PKH                  \
                        --deposit-amount $DEPOSIT_3                  \
                        --deposit-token $DJED                        \
                        --invalid-before $NOW                        \
                        --invalid-hereafter $LATER                   \
                        --marlowe-file revenue-based-loan-07.marlowe \
                        --out-file     revenue-based-loan-08.marlowe \
                        --print-stats


Datum size: 113
Payment 1
  Acccount: PK "84117ffa443e598527726f6509078114ecd1f2c031e71dd5d239d504"
  Payee: Party (PK "84117ffa443e598527726f6509078114ecd1f2c031e71dd5d239d504")
  Ada: 0.000000
  1f4cc97451944ce039366dee5d9acd36a2784dd22e7c3431e50264e8."DjedUSD": 290
Payment 2
  Acccount: PK "e107572dc2e0f7dadaf87c1a4f55d6b5b5456108c1de8f6d75ba0d15"
  Payee: Party (PK "e107572dc2e0f7dadaf87c1a4f55d6b5b5456108c1de8f6d75ba0d15")
  Ada: 2.000000


The final payment to the lender has been made, and the oracle receives back their minimum Ada deposit.

In [46]:
jq .state revenue-based-loan-08.marlowe | json2yaml

accounts: []
boundValues:
- - Remaining Due
  - 290
- - Next Payment
  - 570
choices:
- - choice_name: Revenue
    choice_owner:
      pk_hash: e107572dc2e0f7dadaf87c1a4f55d6b5b5456108c1de8f6d75ba0d15
  - 19000
minTime: 1656878706000


The contract is complete.

In [47]:
jq .contract revenue-based-loan-08.marlowe | json2yaml

close
...


## Run the Contract on Chain

### Find the script address

In [48]:
SCRIPT_ADDRESS=$(jq -r .marloweValidator.address revenue-based-loan-01.marlowe)
echo $SCRIPT_ADDRESS

addr_test1wp3v2mx0ccsh4l6kjtsa86lgnsss20f3lsgcst9jr07axpcmajgcl


### 1. Create the contract

In [49]:
TX_1=$(
marlowe-cli run execute --tx-in "$TX_0_MARLOWE" \
                        --required-signer $MARLOWE_SKEY \
                        --change-address $MARLOWE_ADDR \
                        --marlowe-out-file revenue-based-loan-01.marlowe \
                        --out-file /dev/null \
                        --submit 600 \
                        --print-stats \
| sed -e 's/^TxId "\(.*\)"$/\1/' \
)
echo "TxId \"$TX_1\""


Fee: Lovelace 187061
Size: 477 / 32768 = 1%
Execution units:
  Memory: 0 / 30000000 = 0%
  Steps: 0 / 10000000000 = 0%
TxId "235c2599768473a6249166cb031e0e91139ac12fb7576e02e803fc5adcc3ecd3"


UTxOs at Marlowe validator address:

In [50]:
cardano-cli query utxo --testnet-magic "$CARDANO_TESTNET_MAGIC" --address "$SCRIPT_ADDRESS"

                           TxHash                                 TxIx        Amount
--------------------------------------------------------------------------------------
235c2599768473a6249166cb031e0e91139ac12fb7576e02e803fc5adcc3ecd3     1        2000000 lovelace + TxOutDatumHash ScriptDataInAlonzoEra "a34d01fac6bff27662b2353bc9b34c0f35e73129cd4e5f428e190da027c667ae"


UTxOs at the oracle address:

In [51]:
cardano-cli query utxo --testnet-magic "$CARDANO_TESTNET_MAGIC" --address "$MARLOWE_ADDR"

                           TxHash                                 TxIx        Amount
--------------------------------------------------------------------------------------
235c2599768473a6249166cb031e0e91139ac12fb7576e02e803fc5adcc3ecd3     0        992556672 lovelace + TxOutDatumNone


### 2. The lender deposits the principal

In [52]:
TX_2=$(
marlowe-cli run execute --marlowe-in-file revenue-based-loan-01.marlowe \
                        --tx-in-marlowe "$TX_1#1" \
                        --tx-in-collateral "$TX_0_BEAUMONT_ADA"\
                        --tx-in "$TX_0_BEAUMONT_ADA" \
                        --tx-in "$TX_0_BEAUMONT_DJED" \
                        --required-signer $BEAUMONT_SKEY \
                        --change-address $BEAUMONT_ADDR \
                        --marlowe-out-file revenue-based-loan-02.marlowe \
                        --out-file /dev/null \
                        --submit 600 \
                        --print-stats \
| sed -e 's/^TxId "\(.*\)"$/\1/' \
)
echo "TxId \"$TX_2\""


Fee: Lovelace 1578915
Size: 15235 / 32768 = 46%
Execution units:
  Memory: 8713952 / 30000000 = 29%
  Steps: 3200141318 / 10000000000 = 32%
TxId "8b97a703eea8b8d9d8ebbcd81cc7e2a00edf8a44e054bfbb917835ee8943e2fa"


UTxOs at Marlowe validator address:

In [53]:
cardano-cli query utxo --testnet-magic "$CARDANO_TESTNET_MAGIC" --address "$SCRIPT_ADDRESS"

                           TxHash                                 TxIx        Amount
--------------------------------------------------------------------------------------
8b97a703eea8b8d9d8ebbcd81cc7e2a00edf8a44e054bfbb917835ee8943e2fa     1        2000000 lovelace + TxOutDatumHash ScriptDataInAlonzoEra "eb5dbe9236e0572f5ba23d4582c3b14d22322150f2d3cb36ebac864ac04334f9"


The lender has spent their 1000 `DjedUSD`:

In [54]:
cardano-cli query utxo --testnet-magic "$CARDANO_TESTNET_MAGIC" --address "$BEAUMONT_ADDR"

                           TxHash                                 TxIx        Amount
--------------------------------------------------------------------------------------
8b97a703eea8b8d9d8ebbcd81cc7e2a00edf8a44e054bfbb917835ee8943e2fa     0        976138501 lovelace + TxOutDatumNone
8b97a703eea8b8d9d8ebbcd81cc7e2a00edf8a44e054bfbb917835ee8943e2fa     3        1500000 lovelace + TxOutDatumHash ScriptDataInAlonzoEra "a80c752321932efccbca109a5d1a394dfff6b9c7456ddff8efcafd8933dc7203"


The borrower has received their loan of 1000 `DjedUSD`:

In [55]:
cardano-cli query utxo --testnet-magic "$CARDANO_TESTNET_MAGIC" --address "$KYD_ADDR"

                           TxHash                                 TxIx        Amount
--------------------------------------------------------------------------------------
625b9f53ee6f5f05ca54834b99ca3dceb097be8e934828d353acb2a7e6c74cb7     0        1002574041 lovelace + TxOutDatumNone
8b97a703eea8b8d9d8ebbcd81cc7e2a00edf8a44e054bfbb917835ee8943e2fa     2        1344798 lovelace + 1000 1f4cc97451944ce039366dee5d9acd36a2784dd22e7c3431e50264e8.446a6564555344 + TxOutDatumNone
923e24f6980b547ec3fe76febd97aa050d8d4e52111ed69e9de0ef80a7cd2f93     2        10000000 lovelace + 100 1f4cc97451944ce039366dee5d9acd36a2784dd22e7c3431e50264e8.446a6564555344 + TxOutDatumNone


### 3. The oracle reports the borrower's revenue in the first period

In [56]:
TX_3=$(
marlowe-cli run execute --marlowe-in-file revenue-based-loan-02.marlowe \
                        --tx-in-marlowe "$TX_2#1" \
                        --tx-in-collateral "$TX_1#0" \
                        --tx-in "$TX_1#0" \
                        --required-signer $MARLOWE_SKEY \
                        --change-address $MARLOWE_ADDR \
                        --marlowe-out-file revenue-based-loan-03.marlowe \
                        --out-file /dev/null \
                        --submit 600 \
                        --print-stats \
| sed -e 's/^TxId "\(.*\)"$/\1/' \
)
echo "TxId \"$TX_3\""


Fee: Lovelace 1657057
Size: 16644 / 32768 = 50%
Execution units:
  Memory: 8692696 / 30000000 = 28%
  Steps: 3502737003 / 10000000000 = 35%
TxId "e4e5a0bbae7e884a5ba95d7258d7bf07866d7e3629709ae9cd4a96f4629659cc"


UTxOs at Marlowe validator address:

In [57]:
cardano-cli query utxo --testnet-magic "$CARDANO_TESTNET_MAGIC" --address "$SCRIPT_ADDRESS"

                           TxHash                                 TxIx        Amount
--------------------------------------------------------------------------------------
e4e5a0bbae7e884a5ba95d7258d7bf07866d7e3629709ae9cd4a96f4629659cc     1        2000000 lovelace + TxOutDatumHash ScriptDataInAlonzoEra "c8aef7177cc9be50218a5286d32eabd1c96aebf6b75a3c6de5e349d91fe56aa0"


UTxOs at the oracle address:

In [58]:
cardano-cli query utxo --testnet-magic "$CARDANO_TESTNET_MAGIC" --address "$MARLOWE_ADDR"

                           TxHash                                 TxIx        Amount
--------------------------------------------------------------------------------------
e4e5a0bbae7e884a5ba95d7258d7bf07866d7e3629709ae9cd4a96f4629659cc     0        989399615 lovelace + TxOutDatumNone
e4e5a0bbae7e884a5ba95d7258d7bf07866d7e3629709ae9cd4a96f4629659cc     2        1500000 lovelace + TxOutDatumHash ScriptDataInAlonzoEra "a242745d5603ea853c8b3a113eeb1776cd737d7782f94d451afd0643edc3298e"


### 4. The borrower deposits their first payment.

In [59]:
TX_4=$(
marlowe-cli run execute --marlowe-in-file revenue-based-loan-03.marlowe \
                        --tx-in-marlowe "$TX_3#1" \
                        --tx-in-collateral "$TX_0_KYD_ADA" \
                        --tx-in "$TX_0_KYD_ADA" \
                        --tx-in "$TX_0_KYD_DJED" \
                        --tx-in "$TX_2#2" \
                        --required-signer $KYD_SKEY \
                        --change-address $KYD_ADDR \
                        --marlowe-out-file revenue-based-loan-04.marlowe \
                        --tx-out "$KYD_ADDR+$MIN_ADA+$((PRINCIPAL+INTEREST-DEPOSIT_1)) $DJED" \
                        --out-file /dev/null \
                        --submit 600 \
                        --print-stats \
| sed -e 's/^TxId "\(.*\)"$/\1/' \
)
echo "TxId \"$TX_4\""


Fee: Lovelace 1716047
Size: 15962 / 32768 = 48%
Execution units:
  Memory: 9946716 / 30000000 = 33%
  Steps: 3610263057 / 10000000000 = 36%
TxId "24a9f47301c4e5c1f4de4733a05200c8aa9b6663133812fd837efc8ac18e47ae"


UTxOs at the Marlowe validator address:

In [60]:
cardano-cli query utxo --testnet-magic "$CARDANO_TESTNET_MAGIC" --address "$SCRIPT_ADDRESS"

                           TxHash                                 TxIx        Amount
--------------------------------------------------------------------------------------
24a9f47301c4e5c1f4de4733a05200c8aa9b6663133812fd837efc8ac18e47ae     1        2000000 lovelace + TxOutDatumHash ScriptDataInAlonzoEra "86474f64bbe89ee86aaa6fdc2b9e359c9a22ca816937391d8f242560a59159f9"


The lender has received their payment of 450 `DjedUSD`:

In [61]:
cardano-cli query utxo --testnet-magic "$CARDANO_TESTNET_MAGIC" --address "$BEAUMONT_ADDR"

                           TxHash                                 TxIx        Amount
--------------------------------------------------------------------------------------
24a9f47301c4e5c1f4de4733a05200c8aa9b6663133812fd837efc8ac18e47ae     2        1344798 lovelace + 450 1f4cc97451944ce039366dee5d9acd36a2784dd22e7c3431e50264e8.446a6564555344 + TxOutDatumNone
8b97a703eea8b8d9d8ebbcd81cc7e2a00edf8a44e054bfbb917835ee8943e2fa     0        976138501 lovelace + TxOutDatumNone
8b97a703eea8b8d9d8ebbcd81cc7e2a00edf8a44e054bfbb917835ee8943e2fa     3        1500000 lovelace + TxOutDatumHash ScriptDataInAlonzoEra "a80c752321932efccbca109a5d1a394dfff6b9c7456ddff8efcafd8933dc7203"


The borrower made their payment of 450 `DjedUSD`:

In [62]:
cardano-cli query utxo --testnet-magic "$CARDANO_TESTNET_MAGIC" --address "$KYD_ADDR"

                           TxHash                                 TxIx        Amount
--------------------------------------------------------------------------------------
24a9f47301c4e5c1f4de4733a05200c8aa9b6663133812fd837efc8ac18e47ae     0        1007357994 lovelace + TxOutDatumNone
24a9f47301c4e5c1f4de4733a05200c8aa9b6663133812fd837efc8ac18e47ae     3        2000000 lovelace + 650 1f4cc97451944ce039366dee5d9acd36a2784dd22e7c3431e50264e8.446a6564555344 + TxOutDatumNone
24a9f47301c4e5c1f4de4733a05200c8aa9b6663133812fd837efc8ac18e47ae     4        1500000 lovelace + TxOutDatumHash ScriptDataInAlonzoEra "e193e470b54e3a23da2d15f9bac0a382b88a03c187087cfd28207a2986a03ece"


### 5. The oracle reports the borrower's revenue in the second period

In [63]:
TX_5=$(
marlowe-cli run execute --marlowe-in-file revenue-based-loan-04.marlowe \
                        --tx-in-marlowe "$TX_4#1" \
                        --tx-in-collateral "$TX_3#0" \
                        --tx-in "$TX_3#0" \
                        --tx-in "$TX_3#2" \
                        --required-signer $MARLOWE_SKEY \
                        --change-address $MARLOWE_ADDR \
                        --marlowe-out-file revenue-based-loan-05.marlowe \
                        --out-file /dev/null \
                        --submit 600 \
                        --print-stats \
| sed -e 's/^TxId "\(.*\)"$/\1/' \
)
echo "TxId \"$TX_5\""


Fee: Lovelace 1654274
Size: 16289 / 32768 = 49%
Execution units:
  Memory: 8862038 / 30000000 = 29%
  Steps: 3483614503 / 10000000000 = 34%
TxId "c8c8cb1e17029987474f766fbab9425c579f2ebcb4c312e021a44a70e0f1236f"


UTxOs at the script address:

In [64]:
cardano-cli query utxo --testnet-magic "$CARDANO_TESTNET_MAGIC" --address "$SCRIPT_ADDRESS"

                           TxHash                                 TxIx        Amount
--------------------------------------------------------------------------------------
c8c8cb1e17029987474f766fbab9425c579f2ebcb4c312e021a44a70e0f1236f     1        2000000 lovelace + TxOutDatumHash ScriptDataInAlonzoEra "754c164e802b6a3a4f8e863fb00e33e0f9325e3ec153e50094131f685261224f"


UTxOs at the oracle address:

In [65]:
cardano-cli query utxo --testnet-magic "$CARDANO_TESTNET_MAGIC" --address "$MARLOWE_ADDR"

                           TxHash                                 TxIx        Amount
--------------------------------------------------------------------------------------
c8c8cb1e17029987474f766fbab9425c579f2ebcb4c312e021a44a70e0f1236f     0        987745341 lovelace + TxOutDatumNone
c8c8cb1e17029987474f766fbab9425c579f2ebcb4c312e021a44a70e0f1236f     2        1500000 lovelace + TxOutDatumHash ScriptDataInAlonzoEra "4db7fa7f4725e21c0b61934ac6984540908032ebcfe7de7f43f177dfbf5b4775"


### 6. The borrower deposits their second payment

In [66]:
TX_6=$(
marlowe-cli run execute --marlowe-in-file revenue-based-loan-05.marlowe \
                        --tx-in-marlowe "$TX_5#1" \
                        --tx-in-collateral "$TX_4#0" \
                        --tx-in "$TX_4#0" \
                        --tx-in "$TX_4#3" \
                        --tx-in "$TX_4#4" \
                        --required-signer $KYD_SKEY \
                        --change-address $KYD_ADDR \
                        --marlowe-out-file revenue-based-loan-06.marlowe \
                        --tx-out "$KYD_ADDR+$MIN_ADA+$((PRINCIPAL+INTEREST-DEPOSIT_1-DEPOSIT_2)) $DJED" \
                        --out-file /dev/null \
                        --submit 600 \
                        --print-stats \
| sed -e 's/^TxId "\(.*\)"$/\1/' \
)
echo "TxId \"$TX_6\""


Fee: Lovelace 1666615
Size: 15617 / 32768 = 47%
Execution units:
  Memory: 9567990 / 30000000 = 31%
  Steps: 3438283291 / 10000000000 = 34%
TxId "16a9469bb48dcca6cf18c541971527f8d7cac9931e01d936c26b7f77875aea7a"


UTxOs at Marlowe validator address:

In [67]:
cardano-cli query utxo --testnet-magic "$CARDANO_TESTNET_MAGIC" --address "$SCRIPT_ADDRESS"

                           TxHash                                 TxIx        Amount
--------------------------------------------------------------------------------------
16a9469bb48dcca6cf18c541971527f8d7cac9931e01d936c26b7f77875aea7a     1        2000000 lovelace + TxOutDatumHash ScriptDataInAlonzoEra "9c4c00bd0751ff8fbdbca6cc0ef733a1a0f479736e64d49a036403d94bdee0cf"


The lender has received their payment of 360 `DjedUSD`.

In [68]:
cardano-cli query utxo --testnet-magic "$CARDANO_TESTNET_MAGIC" --address "$BEAUMONT_ADDR"

                           TxHash                                 TxIx        Amount
--------------------------------------------------------------------------------------
16a9469bb48dcca6cf18c541971527f8d7cac9931e01d936c26b7f77875aea7a     2        1344798 lovelace + 360 1f4cc97451944ce039366dee5d9acd36a2784dd22e7c3431e50264e8.446a6564555344 + TxOutDatumNone
24a9f47301c4e5c1f4de4733a05200c8aa9b6663133812fd837efc8ac18e47ae     2        1344798 lovelace + 450 1f4cc97451944ce039366dee5d9acd36a2784dd22e7c3431e50264e8.446a6564555344 + TxOutDatumNone
8b97a703eea8b8d9d8ebbcd81cc7e2a00edf8a44e054bfbb917835ee8943e2fa     0        976138501 lovelace + TxOutDatumNone
8b97a703eea8b8d9d8ebbcd81cc7e2a00edf8a44e054bfbb917835ee8943e2fa     3        1500000 lovelace + TxOutDatumHash ScriptDataInAlonzoEra "a80c752321932efccbca109a5d1a394dfff6b9c7456ddff8efcafd8933dc7203"


The borrower has made their payment of 360 `DjedUSD`.

In [69]:
cardano-cli query utxo --testnet-magic "$CARDANO_TESTNET_MAGIC" --address "$KYD_ADDR"

                           TxHash                                 TxIx        Amount
--------------------------------------------------------------------------------------
16a9469bb48dcca6cf18c541971527f8d7cac9931e01d936c26b7f77875aea7a     0        1004346581 lovelace + TxOutDatumNone
16a9469bb48dcca6cf18c541971527f8d7cac9931e01d936c26b7f77875aea7a     3        2000000 lovelace + 290 1f4cc97451944ce039366dee5d9acd36a2784dd22e7c3431e50264e8.446a6564555344 + TxOutDatumNone
16a9469bb48dcca6cf18c541971527f8d7cac9931e01d936c26b7f77875aea7a     4        1500000 lovelace + TxOutDatumHash ScriptDataInAlonzoEra "f2d6acf8c7504fc265d4a06a574d5c30e4b81441ac9d75ec2d48ec4d4e73a6a7"


### 7. The oracle reports the borrower's revenue in the third period

In [70]:
TX_7=$(
marlowe-cli run execute --marlowe-in-file revenue-based-loan-06.marlowe \
                        --tx-in-marlowe "$TX_6#1" \
                        --tx-in-collateral "$TX_5#0" \
                        --tx-in "$TX_5#0" \
                        --tx-in "$TX_5#2" \
                        --required-signer $MARLOWE_SKEY \
                        --change-address $MARLOWE_ADDR \
                        --marlowe-out-file revenue-based-loan-07.marlowe \
                        --out-file /dev/null \
                        --submit 600 \
                        --print-stats \
| sed -e 's/^TxId "\(.*\)"$/\1/' \
)
echo "TxId \"$TX_7\""


Fee: Lovelace 1580098
Size: 15830 / 32768 = 48%
Execution units:
  Memory: 8266150 / 30000000 = 27%
  Steps: 3211813236 / 10000000000 = 32%
TxId "ef6b4e34d56ff1d34482456ed5bf2311c01e5c3e44d229ff6f9934713e1268b3"


UTxOs at the Marlowe validator address:

In [71]:
cardano-cli query utxo --testnet-magic "$CARDANO_TESTNET_MAGIC" --address "$SCRIPT_ADDRESS"

                           TxHash                                 TxIx        Amount
--------------------------------------------------------------------------------------
ef6b4e34d56ff1d34482456ed5bf2311c01e5c3e44d229ff6f9934713e1268b3     1        2000000 lovelace + TxOutDatumHash ScriptDataInAlonzoEra "d4f72ec5be7709f8dae7323009d1678b41060b5b4eb40a2f28092db37216e0d9"


UTxOs at the oracle address:

In [72]:
cardano-cli query utxo --testnet-magic "$CARDANO_TESTNET_MAGIC" --address "$MARLOWE_ADDR"

                           TxHash                                 TxIx        Amount
--------------------------------------------------------------------------------------
ef6b4e34d56ff1d34482456ed5bf2311c01e5c3e44d229ff6f9934713e1268b3     0        986165243 lovelace + TxOutDatumNone
ef6b4e34d56ff1d34482456ed5bf2311c01e5c3e44d229ff6f9934713e1268b3     2        1500000 lovelace + TxOutDatumHash ScriptDataInAlonzoEra "6aabfb7459cf89ca90c8b91e9d13006ba0b67c5369c2a5f5bce4915b4b26a879"


### 8. The borrower deposits their third payment

In [73]:
TX_8=$(
marlowe-cli run execute --marlowe-in-file revenue-based-loan-07.marlowe \
                        --tx-in-marlowe "$TX_7#1" \
                        --tx-in-collateral "$TX_6#0" \
                        --tx-in "$TX_6#0" \
                        --tx-in "$TX_6#3" \
                        --tx-in "$TX_6#4" \
                        --required-signer $KYD_SKEY \
                        --change-address $KYD_ADDR \
                        --marlowe-out-file revenue-based-loan-08.marlowe \
                        --out-file /dev/null \
                        --submit 600 \
                        --print-stats \
| sed -e 's/^TxId "\(.*\)"$/\1/' \
)
echo "TxId \"$TX_8\""


Fee: Lovelace 1181952
Size: 14148 / 32768 = 43%
Execution units:
  Memory: 4539458 / 30000000 = 15%
  Steps: 1636895683 / 10000000000 = 16%
TxId "836631ef271d86554c717fe12a3feadaf661dafd9a2c80728964244c8b163542"


There is no UTxO at the Marlowe validator address because the contract has closed:

In [74]:
cardano-cli query utxo --testnet-magic "$CARDANO_TESTNET_MAGIC" --address "$SCRIPT_ADDRESS"

                           TxHash                                 TxIx        Amount
--------------------------------------------------------------------------------------


The lender has recieved their payment of 290 `DjedUSD`, completing their receipt of principal and interest.

In [75]:
cardano-cli query utxo --testnet-magic "$CARDANO_TESTNET_MAGIC" --address "$BEAUMONT_ADDR"

                           TxHash                                 TxIx        Amount
--------------------------------------------------------------------------------------
16a9469bb48dcca6cf18c541971527f8d7cac9931e01d936c26b7f77875aea7a     2        1344798 lovelace + 360 1f4cc97451944ce039366dee5d9acd36a2784dd22e7c3431e50264e8.446a6564555344 + TxOutDatumNone
24a9f47301c4e5c1f4de4733a05200c8aa9b6663133812fd837efc8ac18e47ae     2        1344798 lovelace + 450 1f4cc97451944ce039366dee5d9acd36a2784dd22e7c3431e50264e8.446a6564555344 + TxOutDatumNone
836631ef271d86554c717fe12a3feadaf661dafd9a2c80728964244c8b163542     1        1344798 lovelace + 290 1f4cc97451944ce039366dee5d9acd36a2784dd22e7c3431e50264e8.446a6564555344 + TxOutDatumNone
8b97a703eea8b8d9d8ebbcd81cc7e2a00edf8a44e054bfbb917835ee8943e2fa     0        976138501 lovelace + TxOutDatumNone
8b97a703eea8b8d9d8ebbcd81cc7e2a00edf8a44e054bfbb917835ee8943e2fa     3        1500000 lovelace + TxOutDatumHash ScriptDataInAlonzoEra "a80c75232

The borrower has made their payment of 290 `DjedUSD`, completing their obligation.

In [76]:
cardano-cli query utxo --testnet-magic "$CARDANO_TESTNET_MAGIC" --address "$KYD_ADDR"

                           TxHash                                 TxIx        Amount
--------------------------------------------------------------------------------------
836631ef271d86554c717fe12a3feadaf661dafd9a2c80728964244c8b163542     0        1003819831 lovelace + TxOutDatumNone
836631ef271d86554c717fe12a3feadaf661dafd9a2c80728964244c8b163542     3        1500000 lovelace + TxOutDatumHash ScriptDataInAlonzoEra "3d01aec8fc802be7833aa4d7bedc46ec8a7a9ac34de6d5da6167ee8e00fe6604"


The oracle has received back their minimum Ada deposit:

In [77]:
cardano-cli query utxo --testnet-magic "$CARDANO_TESTNET_MAGIC" --address "$MARLOWE_ADDR"

                           TxHash                                 TxIx        Amount
--------------------------------------------------------------------------------------
836631ef271d86554c717fe12a3feadaf661dafd9a2c80728964244c8b163542     2        2000000 lovelace + TxOutDatumNone
ef6b4e34d56ff1d34482456ed5bf2311c01e5c3e44d229ff6f9934713e1268b3     0        986165243 lovelace + TxOutDatumNone
ef6b4e34d56ff1d34482456ed5bf2311c01e5c3e44d229ff6f9934713e1268b3     2        1500000 lovelace + TxOutDatumHash ScriptDataInAlonzoEra "6aabfb7459cf89ca90c8b91e9d13006ba0b67c5369c2a5f5bce4915b4b26a879"


### Clean Up

Clean up the UTxOs so we can easily run this example again.

In [78]:
marlowe-cli transaction simple --tx-in "$TX_2#0" \
                               --tx-in "$TX_2#3" \
                               --tx-in "$TX_4#2" \
                               --tx-in "$TX_6#2" \
                               --tx-in "$TX_8#1" \
                               --required-signer $BEAUMONT_SKEY \
                               --change-address $BEAUMONT_ADDR \
                               --tx-out "$BEAUMONT_ADDR+10000000+1000 $DJED" \
                               --tx-out "$KYD_ADDR+10000000+100 $DJED" \
                               --out-file /dev/null \
                               --submit 600

TxId "5849c47b06918214ca920b82c539b906895a645efcc6a67c9817ce0dcc28eab9"


In [79]:
marlowe-cli transaction simple --tx-in "$TX_8#0" \
                               --tx-in "$TX_8#3" \
                               --required-signer $KYD_SKEY \
                               --change-address $KYD_ADDR \
                               --out-file /dev/null \
                               --submit 600

TxId "e41d27d69dbd4e2a8e053825f0198b42bb4fc1b49ca5ec97421ed8956ee63b2b"


In [80]:
marlowe-cli transaction simple --tx-in "$TX_7#0" \
                               --tx-in "$TX_7#2" \
                               --tx-in "$TX_8#2" \
                               --required-signer $MARLOWE_SKEY \
                               --change-address $MARLOWE_ADDR \
                               --out-file /dev/null \
                               --submit 600

TxId "acf9664e4fddd78e827b36289d96914a664e3505ffb72061744f9eac8fe087af"
